/opt/anaconda3/envs/bioinformatica_py/lib/python3.12/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


ModuleNotFoundError: No module named 'pandas'

In [2]:
conda install pandas

Retrieving notices: done
Channels:
 - defaults
 - conda-forge
Platform: osx-arm64
Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3/envs/bioinformatica_py

  added / updated specs:
    - pandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bottleneck-1.4.2           |  py312ha86b861_0         124 KB
    numexpr-2.10.1             |  py312h5d9532f_0         181 KB
    pandas-2.2.3               |  py312hcf29cfe_0        14.6 MB
    python-tzdata-2025.2       |     pyhd3eb1b0_0         141 KB
    ------------------------------------------------------------
                                           Total:        15.1 MB

The following NEW packages will be INSTALLED:

  bottleneck         pkgs/main/osx-arm64::bottleneck-1.4.2-py312ha86b861_0 
  numexpr            pkgs/main/osx-arm64::numexpr-2.10.1-py312h5d9532f_0 
  pandas             pkgs/main/osx-a

In [9]:
from Bio import Entrez, SeqIO
from Bio import pairwise2
import pandas as pd
import time
import os
from urllib.error import HTTPError, URLError
import re

# Configuración de NCBI
Entrez.email = "tu_email@dominio.com"  # Usa tu email real
Entrez.api_key = None  # Considera obtener una API key para más consultas
Entrez.sleep_between_tries = 10  # Tiempo de espera entre intentos

# Virus de referencia: Virus del Mosaico del Tabaco (TMV)
REFERENCE_VIRUS_ID = "NC_001367.1"  # TMV genoma completo
REFERENCE_SEQUENCE = None

def download_reference_sequence():
    """Descarga la secuencia de referencia para TMV"""
    global REFERENCE_SEQUENCE
    max_retries = 3
    for attempt in range(max_retries):
        try:
            print(f"Descargando secuencia referencia TMV (Intento {attempt + 1})...")
            with Entrez.efetch(db="nucleotide", id=REFERENCE_VIRUS_ID, rettype="fasta", retmode="text") as handle:
                REFERENCE_SEQUENCE = SeqIO.read(handle, "fasta")
            print(f"✅ Secuencia de referencia {REFERENCE_VIRUS_ID} descargada")
            print(f"Descripción: {REFERENCE_SEQUENCE.description}")
            print(f"Longitud: {len(REFERENCE_SEQUENCE)} nucleótidos")
            return True
        except HTTPError as e:
            print(f"⚠️ Error HTTP (intento {attempt + 1}/{max_retries}): {e.code}")
            if attempt == max_retries - 1:
                print("❌ Fallo al descargar secuencia de referencia")
                return False
            time.sleep(5)
        except Exception as e:
            print(f"❌ Error inesperado: {str(e)}")
            return False

def calculate_similarity(seq1, seq2):
    """Calcula porcentaje de identidad entre secuencias de TMV"""
    try:
        # Alineamiento global optimizado para virus RNA
        alignments = pairwise2.align.globalms(
            seq1, seq2,
            2, -1, -2, -1,  # match, mismatch, open gap, extend gap
            one_alignment_only=True
        )
        
        if not alignments:
            print("⚠️ No se pudo alinear las secuencias")
            return 0.0
            
        best = alignments[0]
        matches = sum(a == b for a, b in zip(best.seqA, best.seqB))
        return (matches / max(len(seq1), len(seq2))) * 100
    except Exception as e:
        print(f"⚠️ Error cálculo similitud: {str(e)}")
        return 0.0

def buscar_descargar_tmv():
    """Busca y descarga secuencias de TMV de NCBI"""
    if not download_reference_sequence():
        return

    # Búsqueda optimizada para TMV (genoma RNA ~6.4 kb)
    query = "(Tobacco mosaic virus[Organism] OR TMV[Title]) AND complete genome[Title] AND 6000:7000[SLEN]"
    print(f"\n🔍 Búsqueda NCBI: {query}")
    
    try:
        # Búsqueda con manejo de errores
        for attempt in range(3):
            try:
                handle = Entrez.esearch(
                    db="nucleotide",
                    term=query,
                    retmax=15,  # Limite de resultados
                    usehistory="y"
                )
                search_results = Entrez.read(handle)
                handle.close()
                break
            except Exception as e:
                if attempt == 2:
                    raise
                print(f"⚠️ Error búsqueda (intento {attempt + 1}): {str(e)}")
                time.sleep(15)
        
        accession_ids = search_results["IdList"]
        print(f"📊 Secuencias encontradas: {len(accession_ids)}")
        
        if not accession_ids:
            print("❌ No se encontraron secuencias de TMV")
            return

        # Directorio para resultados
        os.makedirs("tmv_sequences", exist_ok=True)
        metadata = []
        
        # Procesamiento de secuencias
        for i, acc_id in enumerate(accession_ids, 1):
            try:
                print(f"\n🌀 Procesando {i}/{len(accession_ids)}: {acc_id}")
                
                # Descarga con reintentos
                seq_record = None
                for attempt in range(3):
                    try:
                        with Entrez.efetch(db="nucleotide", id=acc_id, rettype="fasta", retmode="text") as handle:
                            seq_record = SeqIO.read(handle, "fasta")
                        break
                    except Exception as e:
                        if attempt == 2:
                            raise
                        print(f"⚠️ Error descarga (intento {attempt + 1}): {str(e)}")
                        time.sleep(10)
                
                # Guardar secuencia
                output_file = f"tmv_sequences/{acc_id}.fasta"
                with open(output_file, "w") as f:
                    SeqIO.write(seq_record, f, "fasta")
                
                # Calcular similitud
                similarity = calculate_similarity(REFERENCE_SEQUENCE.seq, seq_record.seq)
                
                # Extraer metadatos
                desc = seq_record.description
                host = "Desconocido"
                if "[" in desc:
                    host = desc.split("[")[-1].split("]")[0]
                
                # Identificar variante si está en la descripción
                variant = "Común"
                if "variant" in desc.lower():
                    variant = re.search(r'variant\s+([\w-]+)', desc, re.IGNORECASE)
                    variant = variant.group(1) if variant else "Variante"
                
                metadata.append({
                    "ID": seq_record.id,
                    "Archivo": output_file,
                    "Variante": variant,
                    "Huésped": host,
                    "Longitud": len(seq_record),
                    "Similitud (%)": round(similarity, 2)
                })
                
                print(f"✅ {seq_record.id} | Variante: {variant} | Similitud: {similarity:.2f}%")
                
                time.sleep(2)  # Cumplir políticas NCBI
                
            except Exception as e:
                print(f"❌ Error procesando {acc_id}: {str(e)}")
                continue

        # Guardar resultados
        if metadata:
            df = pd.DataFrame(metadata)
            df = df.sort_values("Similitud (%)", ascending=False)
            
            # Análisis rápido
            print("\n📊 Resumen de resultados:")
            print(df[["ID", "Variante", "Similitud (%)", "Longitud"]])
            
            # Guardar CSV
            df.to_csv("tmv_metadata.csv", index=False)
            print("\n💾 Resultados guardados en:")
            print(f"- Secuencias: tmv_sequences/")
            print(f"- Metadatos: tmv_metadata.csv")
            
            # Estadísticas adicionales
            print("\n📈 Estadísticas:")
            print(f"• Secuencias procesadas: {len(df)}")
            print(f"• Longitud promedio: {df['Longitud'].mean():.0f} bp")
            print(f"• Similitud promedio: {df['Similitud (%)'].mean():.2f}%")
            print(f"• Variantes únicas: {df['Variante'].nunique()}")
    
    except Exception as e:
        print(f"❌ Error fatal: {str(e)}")

if __name__ == "__main__":
    print("=== Análisis del Virus del Mosaico del Tabaco (TMV) ===")
    print("=== Genoma ARN de ~6.4 kb - Uno de los más sencillos ===")
    buscar_descargar_tmv()

=== Análisis del Virus del Mosaico del Tabaco (TMV) ===
=== Genoma ARN de ~6.4 kb - Uno de los más sencillos ===
Descargando secuencia referencia TMV (Intento 1)...
⚠️ Error HTTP (intento 1/3): 500
Descargando secuencia referencia TMV (Intento 2)...
⚠️ Error HTTP (intento 2/3): 500
Descargando secuencia referencia TMV (Intento 3)...
⚠️ Error HTTP (intento 3/3): 500
❌ Fallo al descargar secuencia de referencia


In [7]:
from Bio.Blast.Applications import NcbimakeblastdbCommandline, NcbiblastnCommandline
from Bio import SeqIO
import os
import subprocess

def crear_blast_db(archivo_fasta, tipo_db='nucl', titulo="MiDB"):
    """
    Crea una base de datos BLAST a partir de un archivo FASTA
    
    Args:
        archivo_fasta (str): Ruta al archivo FASTA de entrada
        tipo_db (str): 'nucl' para nucleótidos, 'prot' para proteínas
        titulo (str): Título para la base de datos
    
    Returns:
        str: Ruta al directorio de la base de datos creada
    """
    # Verificar si el archivo FASTA existe
    if not os.path.isfile(archivo_fasta):
        raise FileNotFoundError(f"El archivo {archivo_fasta} no existe")
    
    # Crear directorio para la base de datos si no existe
    db_dir = os.path.join(os.path.dirname(archivo_fasta), "blast_db")
    os.makedirs(db_dir, exist_ok=True)
    
    # Nombre base para los archivos de la DB (sin extensión)
    db_name = os.path.join(db_dir, os.path.splitext(os.path.basename(archivo_fasta))[0])
    
    # Comando para crear la base de datos
    makeblastdb_cline = NcbimakeblastdbCommandline(
        input_file=archivo_fasta,
        dbtype=tipo_db,
        title=titulo,
        out=db_name,
        parse_seqids=True  # Permite búsqueda por identificadores de secuencia
    )
    
    print(f"Creando base de datos BLAST desde {archivo_fasta}...")
    print("Comando ejecutado:", makeblastdb_cline)
    
    # Ejecutar el comando
    stdout, stderr = makeblastdb_cline()
    
    # Verificar errores
    if stderr:
        print("Errores durante la creación de la DB:")
        print(stderr)
    else:
        print(f"✅ Base de datos creada exitosamente en {db_dir}")
    
    return db_dir

def ejecutar_blast_local(consulta, db_path, tipo_consulta='nucl', evalue=0.001, outfmt=5, out_file=None):
    """
    Ejecuta una búsqueda BLAST contra una base de datos local
    
    Args:
        consulta: Puede ser una secuencia (str), un archivo FASTA o un objeto SeqRecord
        db_path: Ruta al directorio con la base de datos BLAST
        tipo_consulta: 'nucl' o 'prot' según el tipo de consulta
        evalue: Umbral de e-value
        outfmt: Formato de salida (5 para XML, otros valores ver documentación BLAST)
        out_file: Archivo de salida (si None, devuelve los resultados)
    
    Returns:
        Resultados BLAST o None si se especificó archivo de salida
    """
    # Determinar archivos de la base de datos (tomamos el primer .nhr que encontremos)
    db_files = [f for f in os.listdir(db_path) if f.endswith('.nhr') or f.endswith('.phr')]
    if not db_files:
        raise ValueError(f"No se encontraron archivos de base de datos BLAST en {db_path}")
    
    db_name = os.path.join(db_path, os.path.splitext(db_files[0])[0])
    
    # Preparar la consulta
    if isinstance(consulta, str) and os.path.isfile(consulta):
        # Es un archivo FASTA
        query = consulta
    else:
        # Es una secuencia directa o objeto SeqRecord
        if hasattr(consulta, 'seq'):  # Si es un objeto SeqRecord
            seq = str(consulta.seq)
        else:
            seq = consulta
        
        # Crear archivo temporal con la consulta
        query = os.path.join(db_path, "temp_query.fasta")
        with open(query, "w") as f:
            f.write(f">temp_query\n{seq}")
    
    # Determinar el tipo de BLAST a usar
    if tipo_consulta == 'nucl':
        blast_cline = NcbiblastnCommandline
    elif tipo_consulta == 'prot':
        blast_cline = NcbiblastpCommandline
    else:
        raise ValueError("tipo_consulta debe ser 'nucl' o 'prot'")
    
    # Configurar el comando BLAST
    if out_file is None:
        out_file = os.path.join(db_path, "blast_results.xml")
    
    blast_cline = blast_cline(
        query=query,
        db=db_name,
        evalue=evalue,
        out=out_file,
        outfmt=outfmt
    )
    
    print(f"Ejecutando BLAST contra la base de datos {db_name}...")
    print("Comando ejecutado:", blast_cline)
    
    # Ejecutar BLAST
    stdout, stderr = blast_cline()
    
    if stderr:
        print("Errores durante la ejecución de BLAST:")
        print(stderr)
    
    # Leer resultados si no se especificó archivo de salida
    if out_file:
        print(f"✅ Resultados guardados en {out_file}")
        return None
    else:
        from Bio.Blast import NCBIXML
        with open(out_file) as result_handle:
            blast_records = NCBIXML.parse(result_handle)
            return list(blast_records)

def analizar_resultados_blast(blast_records, umbral_identidad=70.0):
    """
    Analiza los resultados de BLAST y muestra los hits significativos
    
    Args:
        blast_records: Lista de objetos BlastRecord
        umbral_identidad: Porcentaje mínimo de identidad para considerar un hit
    """
    print("\nAnálisis de resultados BLAST:")
    
    for blast_record in blast_records:
        for alignment in blast_record.alignments:
            for hsp in alignment.hsps:
                identidad = (hsp.identities / hsp.align_length) * 100
                if identidad >= umbral_identidad:
                    print(f"\nHit significativo encontrado:")
                    print(f"Secuencia: {alignment.title}")
                    print(f"Longitud: {alignment.length}")
                    print(f"Identidad: {identidad:.2f}%")
                    print(f"E-value: {hsp.expect}")
                    print(f"Aliniamiento:\n{hsp.query[0:75]}...")
                    print(f"           {hsp.match[0:75]}...")
                    print(f"           {hsp.sbjct[0:75]}...")

def ejemplo_completo():
    # 1. Crear base de datos desde un archivo FASTA
    archivo_fasta = "tu_archivo.fasta"  # Reemplaza con tu archivo FASTA
    db_dir = crear_blast_db(archivo_fasta)
    
    # 2. Ejemplo de secuencia de consulta (puede ser un archivo FASTA o una secuencia directa)
    secuencia_consulta = "ATGCGTACGTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGC"
    
    # 3. Ejecutar BLAST
    resultados = ejecutar_blast_local(secuencia_consulta, db_dir, out_file="resultados_blast.xml")
    
    # 4. Analizar resultados (si no se especificó out_file)
    if resultados:
        analizar_resultados_blast(resultados)
    else:
        # Leer resultados desde el archivo
        from Bio.Blast import NCBIXML
        with open("resultados_blast.xml") as result_handle:
            blast_records = NCBIXML.parse(result_handle)
            analizar_resultados_blast(blast_records)

if __name__ == "__main__":
    ejemplo_completo()
    

/opt/anaconda3/envs/bioinformatica_py/lib/python3.12/site-packages/Bio/Application/__init__.py:39: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(


FileNotFoundError: El archivo tu_archivo.fasta no existe